In [2]:
cd NoahSim/

/Users/noahkasmanoff/BurstCube/Simulation/Localization_and_Detection/NoahSim


In [3]:
import GRBgenerator

Below is the fastcube class, simulations for burstcube that run really fast but still pretty accurate. What sorts of customizations should I include?

Sample, samples n, 

Keep that test feature: 
maybe have a bruteforce vs. quicker version, all about fullsky sims now though. 

Also need to remember how I treated past horizon spots, and what the goal should be for those. 

In [32]:
#The following cell contains the "FastCube" class. This is the simulation I hope to use to be able to run quicker simulations. 

import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s
import time as time

class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = (float(input("Please enter the second tilt (deg) ")))
            self.tiltB = np.deg2rad(self.tiltB)
            self.tiltC = self.tiltA = np.deg2rad(dettilt)
            self.tiltD = self.tiltB
        
        self.zenith = [0 , 0]
        self.bg = background


        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, test=True):   #is this how I inherit? 
        start = time.time()
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        

        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        """
        
        if test:
            sample = 1
            samples = 500  #times  per sky pos
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 0
            topA = 1000
            ntheta = 10   #over sky chi points
            nphi = 37
            nA = 100

        else:
            sample = len(GRB.sourceangs) 
            samples = 10 #times  per sky pos
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 400
            topA = 1000
            ntheta = 31   #over sky chi points
            nphi = 120
            nA = 12
        self.localizationerrors = []    
        self.X_all = []
        self.y_all = []
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
          # print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:
                sepA=bf.angle(sourcexyz,self.normA)
                   # print("separation from A is " + str(np.rad2deg(sepA)))
                   #this check passes.  
               
                if sepA < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryA=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normA))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryA = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsA = dtheoryA + self.bg #another artifact, incl this background effect somewhere
                unccountsA = np.sqrt(countsA)
                detactualA = rand.gauss(countsA,unccountsA)  #there is a lot of noise, present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = detactualA
                
                sepB=bf.angle(sourcexyz,self.normB)
                   # print("separation from B is " + str(np.rad2deg(sepB)))
                   #this check passes.  
               
                if sepB < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryB=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normB))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryB = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsB = dtheoryB + self.bg #another artifact, incl this background effect somewhere
                unccountsB = np.sqrt(countsB)
                detactualB = rand.gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = detactualB

                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation from C is " + str(np.rad2deg(sepC)))
                   #this check passes.  
               
                if sepC < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryC=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normC))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryC = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = np.sqrt(countsC)
                detactualC = rand.gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = detactualC
                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation from D is " + str(np.rad2deg(sepD)))
                   #this check passes.  
               
                if sepD < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryD=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normD))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryD = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = np.sqrt(countsD)
                detactualD = rand.gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = detactualD
                
                self.X_all.append([detcountsA,detcountsB,detcountsC,detcountsD])
                #coarse to fine optimization
                chiA = bf.quad_solver(detcountsA,self.normA,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiB = bf.quad_solver(detcountsB,self.normB,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiC = bf.quad_solver(detcountsC,self.normC,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiD = bf.quad_solver(detcountsD,self.normD,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                
                chisquared = np.add(np.add(chiA,chiB),np.add(chiC,chiD)) #adds it all up for total chi2
                
                #print("Chi squareds: " +str(chisquared))
                
                
                thetaloc, philoc, Aguess = bf.indexer(chisquared,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA)
                recvec = hp.ang2vec(np.deg2rad(thetaloc),np.deg2rad(philoc))
                locoffset = np.rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                self.y_all.append([thetaloc,philoc,Aguess])
                locunc.append(locoffset)
                loop +=1
            #print("Avg loc offset = " + str(s.mean(locunc)) + " deg.")
            self.localizationerrors.append(s.mean(locunc))
        return self.localizationerrors

In [33]:
GRBtest = GRBgenerator.Sky(2,500)

In [34]:
import time

In [35]:
fastguy = FastCube(1000,45,alternating=False)

In [38]:
start = time.time()
fastguy.response2GRB(GRBtest,test=True)
end= time.time()

print(end-start)

9.326045274734497


In [39]:
len(fastguy.y_all)

500

Now the NN Part!

In [40]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [41]:
xall = np.array(fastguy.X_all)

yall = np.array(fastguy.y_all)

In [42]:
from sklearn.neural_network import MLPRegressor

In [43]:
clf = MLPRegressor()


In [44]:
clf.hidden_layer_sizes=5000
clf.n_layers_=5000
clf.verbose=1

In [45]:
clf.fit(xall,yall)


Iteration 1, loss = 15425.21759749
Iteration 2, loss = 11023.03737243
Iteration 3, loss = 1295.68627490
Iteration 4, loss = 5070.94366816
Iteration 5, loss = 698.11614350
Iteration 6, loss = 2270.23425849
Iteration 7, loss = 1028.88388171
Iteration 8, loss = 696.77050278
Iteration 9, loss = 1043.42306914
Iteration 10, loss = 210.23624991
Iteration 11, loss = 667.54504773
Iteration 12, loss = 225.52613522
Iteration 13, loss = 350.79236286
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=5000, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=1, warm_start=False)

In [46]:
clf.predict(np.array([[1445.15726843, 1464.58808801, 1350.0381453 , 1245.37103302]]))

array([[ 21.65692853,  45.95874145, 473.16597437]])

array([1445.15726843, 1464.58808801, 1350.0381453 , 1245.37103302])

In [19]:
yall[0]

array([ 21.        ,  66.55462185, 509.09090909])

In [21]:
np.rad2deg(GRBtest.sourceangs)

array([[ 23.55646431,  45.        ],
       [ 23.55646431, 135.        ],
       [ 23.55646431, 225.        ],
       [ 23.55646431, 315.        ],
       [ 48.1896851 ,  22.5       ],
       [ 48.1896851 ,  67.5       ],
       [ 48.1896851 , 112.5       ],
       [ 48.1896851 , 157.5       ],
       [ 48.1896851 , 202.5       ],
       [ 48.1896851 , 247.5       ],
       [ 48.1896851 , 292.5       ],
       [ 48.1896851 , 337.5       ],
       [ 70.52877937,   0.        ],
       [ 70.52877937,  45.        ],
       [ 70.52877937,  90.        ],
       [ 70.52877937, 135.        ],
       [ 70.52877937, 180.        ],
       [ 70.52877937, 225.        ],
       [ 70.52877937, 270.        ],
       [ 70.52877937, 315.        ],
       [ 90.        ,  22.5       ],
       [ 90.        ,  67.5       ],
       [ 90.        , 112.5       ],
       [ 90.        , 157.5       ],
       [ 90.        , 202.5       ],
       [ 90.        , 247.5       ],
       [ 90.        , 292.5       ],
 

In [49]:
np.rad2deg(GRBtest.sourceangs)

array([[ 23.55646431,  45.        ],
       [ 23.55646431, 135.        ],
       [ 23.55646431, 225.        ],
       [ 23.55646431, 315.        ],
       [ 48.1896851 ,  22.5       ],
       [ 48.1896851 ,  67.5       ],
       [ 48.1896851 , 112.5       ],
       [ 48.1896851 , 157.5       ],
       [ 48.1896851 , 202.5       ],
       [ 48.1896851 , 247.5       ],
       [ 48.1896851 , 292.5       ],
       [ 48.1896851 , 337.5       ],
       [ 70.52877937,   0.        ],
       [ 70.52877937,  45.        ],
       [ 70.52877937,  90.        ],
       [ 70.52877937, 135.        ],
       [ 70.52877937, 180.        ],
       [ 70.52877937, 225.        ],
       [ 70.52877937, 270.        ],
       [ 70.52877937, 315.        ],
       [ 90.        ,  22.5       ],
       [ 90.        ,  67.5       ],
       [ 90.        , 112.5       ],
       [ 90.        , 157.5       ],
       [ 90.        , 202.5       ],
       [ 90.        , 247.5       ],
       [ 90.        , 292.5       ],
 

Moral of the story: 
This ANN concept works if the net is huge, and you're starting if off with very small sample regions, otherwise its a waste of time. Loss is going to be big always too. Cool proof of concept above, but not seemingly appliciable in pracrtice unless I merge together a net for every single sky position for every different possible detector orientation which is absurd.